In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/IMDB.csv')

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(50000, 2)

In [ ]:
!pip install tensorflow


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from bs4 import BeautifulSoup

In [ ]:
def preprocess(text):
    # Convert to lowercase
    text_lower = text.lower()

    # Remove HTML tags
    soup = BeautifulSoup(text_lower, 'html.parser')
    clean_text = soup.get_text()

    return clean_text

In [ ]:
df['review'] = df['review'].apply(preprocess)
df['review']

,review
0,one of the other reviewers has mentioned that ...
1,a wonderful little production. the filming tec...
2,i thought this was a wonderful way to spend ti...
3,basically there's a family where a little boy ...
4,"petter mattei's ""love in the time of money"" is..."
...,...
49995,i thought this movie did a down right good job...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,i am a catholic taught in parochial elementary...
49998,i'm going to have to disagree with the previou...


In [ ]:
def convert(text,tokenizer,max_len=500):
      # Tokenize the text into a sequence of integers
    seq = tokenizer.texts_to_sequences([text])

    # Pad sequences to ensure uniform length
    padded_seq = pad_sequences(seq, maxlen=max_len)
    return padded_seq

In [ ]:
tokenizer = Tokenizer(num_words=10000)  #limit to top 10,000 words
tokenizer.fit_on_texts(df['review'])

df['review'] = df['review'].apply(lambda x: convert(x, tokenizer))

500

In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
X = np.array(df['review'].tolist())
y=df['sentiment']
y = df['sentiment'].map({'positive': 0, 'negative': 1})

In [ ]:
X

array([[[   0,    0,    0, ...,  124, 4102,  485]],

       [[   0,    0,    0, ..., 1975,   68,  220]],

       [[   0,    0,    0, ...,   62,   15,  349]],

       ...,

       [[   0,    0,    0, ...,    2,    2, 6043]],

       [[   0,    0,    0, ...,   66,  738,   40]],

       [[   0,    0,    0, ...,  792,   10,   16]]], dtype=int32)

In [ ]:
y

,sentiment
0,0
1,0
2,0
3,1
4,0
...,...
49995,0
49996,1
49997,1
49998,1


In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
#Model buliding
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Embedding,Flatten

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=10000,output_dim=16,output_length=500))
model.add(Flatten())
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



In [ ]:
HISTORY = model.fit(X_train, y_train, validation_split = 0.3, epochs = 20, batch_size = 64)


Epoch 1/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 1.0000 - loss: 4.3124e-04 - val_accuracy: 0.8674 - val_loss: 0.6903
Epoch 2/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 1.0000 - loss: 3.0075e-04 - val_accuracy: 0.8676 - val_loss: 0.7064
Epoch 3/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 1.0000 - loss: 2.3557e-04 - val_accuracy: 0.8677 - val_loss: 0.7356
Epoch 4/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 1.0000 - loss: 1.5002e-04 - val_accuracy: 0.8677 - val_loss: 0.7962
Epoch 5/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 1.0000 - loss: 6.9053e-05 - val_accuracy: 0.8672 - val_loss: 0.8201
Epoch 6/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 1.0000 - loss: 5.4331e-05 - val_accuracy: 0.8673 - val_loss: 0.8330
Epoch 7/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 1.0000 - loss: 4.4591e-05 - val_accuracy: 0.8673 - val_loss: 0.8486
Epoch 8/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 1.000

In [ ]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
from sklearn.metrics import confusion_matrix ,accuracy_score
cm = confusion_matrix(y_test, predictions)
cm

array([[4306,  650],
       [ 652, 4392]])

In [ ]:
accuracy_score(y_test,predictions)

0.8698